In [279]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


from warnings import filterwarnings
filterwarnings('ignore')

In [280]:
train_dataframe = pd.read_csv("data/space_train.csv")

In [281]:
train_dataframe

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [285]:
int(df['Cabin2'].median())


407

In [221]:
train_dataframe.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [286]:
def preprocess_data(df):
    df = df.copy()
    
    def delete_id(df):
        df = df.copy()
        df.drop(columns="PassengerId", inplace=True)
        return df
    
    def Cabin(df):
        df = df.copy()
        new_columns = df['Cabin'].str.split('/', expand=True)
        new_columns.columns = ['Cabin1', 'Cabin2', 'Cabin3']
        df = pd.concat([df, new_columns], axis=1)
        df.drop(columns="Cabin", inplace=True)
        return df  
    
    def Name(df):
        df = df.copy()
        df.drop(columns="Name", inplace=True)
        return df
    
    
    df = delete_id(df)
    df = Cabin(df)
    df = Name(df)
    df['VIP'].fillna(False, inplace=True)
    df['HomePlanet'].fillna('Earth', inplace=True)
    df['CryoSleep'].fillna(False, inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['RoomService'].fillna(df['RoomService'].mean(), inplace=True)
    indexies = df[df['Cabin1'].isna()].index
    ind = df[df['Destination'].isna()].index
    df['Cabin1'].values[indexies] = np.array([df['Cabin1'].mode()] * len(indexies)).squeeze()
    df['Cabin2'].values[indexies] = np.array([int(df['Cabin2'].median())] * len(indexies)).squeeze()
    df['Cabin3'].values[indexies] = np.array([df['Cabin3'].mode()] * len(indexies)).squeeze()
    df['Destination'].values[ind] = np.array([df['Destination'].mode()] * len(ind)).squeeze()
    df['Spa'].fillna(df['Spa'].mean(), inplace=True)
    df['VRDeck'].fillna(df['VRDeck'].mean(), inplace=True)
    df['FoodCourt'].fillna(df['FoodCourt'].mean(), inplace=True)
    df['ShoppingMall'].fillna(df['ShoppingMall'].mean(), inplace=True)

    return df

In [320]:
df = preprocess_data(train_dataframe)


df['Destination']

0         TRAPPIST-1e
1         TRAPPIST-1e
2         TRAPPIST-1e
3         TRAPPIST-1e
4         TRAPPIST-1e
            ...      
8688      55 Cancri e
8689    PSO J318.5-22
8690      TRAPPIST-1e
8691      55 Cancri e
8692      TRAPPIST-1e
Name: Destination, Length: 8693, dtype: object

In [288]:
from catboost import  CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [289]:
num_cols = [
    'RoomService',
    'FoodCourt', 
    'ShoppingMall', 
    'Spa', 
    'VRDeck',
    'Age'
]

target = ['Transported']

cat_cols = list(set(df.columns) - set(num_cols) - set(target))
feat = cat_cols + num_cols
cat_cols

['Cabin1', 'VIP', 'Cabin2', 'Destination', 'CryoSleep', 'HomePlanet', 'Cabin3']

In [302]:

X_train, X_test, Y_train, Y_test = train_test_split(df[feat], 
                                                    df[target],
                                                   train_size=0.9)


In [310]:
ctb = CatBoostClassifier(n_estimators=2000)
ctb.fit(X_train, Y_train, cat_features=cat_cols)
pred = ctb.predict_proba(X_test)[:, 1]


Learning rate set to 0.013133
0:	learn: 0.6863052	total: 6.69ms	remaining: 13.4s
1:	learn: 0.6803792	total: 11.6ms	remaining: 11.6s
2:	learn: 0.6741183	total: 16.4ms	remaining: 10.9s
3:	learn: 0.6676348	total: 20.9ms	remaining: 10.4s
4:	learn: 0.6617998	total: 25.9ms	remaining: 10.3s
5:	learn: 0.6557894	total: 30.2ms	remaining: 10s
6:	learn: 0.6500242	total: 34.3ms	remaining: 9.76s
7:	learn: 0.6446418	total: 38.7ms	remaining: 9.62s
8:	learn: 0.6397376	total: 43ms	remaining: 9.51s
9:	learn: 0.6346681	total: 46.9ms	remaining: 9.34s
10:	learn: 0.6299566	total: 50.9ms	remaining: 9.2s
11:	learn: 0.6251068	total: 55.7ms	remaining: 9.23s
12:	learn: 0.6208473	total: 60ms	remaining: 9.16s
13:	learn: 0.6165906	total: 64.5ms	remaining: 9.15s
14:	learn: 0.6113132	total: 68.7ms	remaining: 9.09s
15:	learn: 0.6073836	total: 72.8ms	remaining: 9.03s
16:	learn: 0.6029024	total: 76.8ms	remaining: 8.96s
17:	learn: 0.5992481	total: 81.4ms	remaining: 8.96s
18:	learn: 0.5951630	total: 85.7ms	remaining: 8.93s

172:	learn: 0.4324504	total: 795ms	remaining: 8.4s
173:	learn: 0.4321354	total: 800ms	remaining: 8.4s
174:	learn: 0.4317911	total: 805ms	remaining: 8.39s
175:	learn: 0.4314532	total: 810ms	remaining: 8.39s
176:	learn: 0.4312356	total: 815ms	remaining: 8.39s
177:	learn: 0.4309735	total: 820ms	remaining: 8.39s
178:	learn: 0.4307373	total: 824ms	remaining: 8.38s
179:	learn: 0.4306937	total: 827ms	remaining: 8.36s
180:	learn: 0.4304213	total: 832ms	remaining: 8.36s
181:	learn: 0.4302600	total: 836ms	remaining: 8.35s
182:	learn: 0.4300827	total: 840ms	remaining: 8.34s
183:	learn: 0.4297702	total: 845ms	remaining: 8.34s
184:	learn: 0.4294605	total: 849ms	remaining: 8.33s
185:	learn: 0.4292050	total: 854ms	remaining: 8.33s
186:	learn: 0.4289760	total: 859ms	remaining: 8.33s
187:	learn: 0.4287256	total: 864ms	remaining: 8.33s
188:	learn: 0.4285057	total: 868ms	remaining: 8.32s
189:	learn: 0.4282581	total: 872ms	remaining: 8.31s
190:	learn: 0.4278308	total: 877ms	remaining: 8.3s
191:	learn: 0.4

349:	learn: 0.4061657	total: 1.6s	remaining: 7.53s
350:	learn: 0.4061101	total: 1.6s	remaining: 7.52s
351:	learn: 0.4060691	total: 1.6s	remaining: 7.52s
352:	learn: 0.4059954	total: 1.61s	remaining: 7.51s
353:	learn: 0.4058739	total: 1.61s	remaining: 7.51s
354:	learn: 0.4057938	total: 1.62s	remaining: 7.5s
355:	learn: 0.4056322	total: 1.62s	remaining: 7.5s
356:	learn: 0.4055255	total: 1.63s	remaining: 7.49s
357:	learn: 0.4054188	total: 1.63s	remaining: 7.49s
358:	learn: 0.4052733	total: 1.64s	remaining: 7.48s
359:	learn: 0.4051914	total: 1.64s	remaining: 7.48s
360:	learn: 0.4051335	total: 1.65s	remaining: 7.47s
361:	learn: 0.4050878	total: 1.65s	remaining: 7.47s
362:	learn: 0.4049727	total: 1.66s	remaining: 7.47s
363:	learn: 0.4048686	total: 1.66s	remaining: 7.46s
364:	learn: 0.4048203	total: 1.67s	remaining: 7.46s
365:	learn: 0.4047268	total: 1.67s	remaining: 7.45s
366:	learn: 0.4046290	total: 1.67s	remaining: 7.45s
367:	learn: 0.4045622	total: 1.68s	remaining: 7.45s
368:	learn: 0.404

528:	learn: 0.3942753	total: 2.4s	remaining: 6.66s
529:	learn: 0.3942411	total: 2.4s	remaining: 6.66s
530:	learn: 0.3941781	total: 2.41s	remaining: 6.66s
531:	learn: 0.3941144	total: 2.42s	remaining: 6.67s
532:	learn: 0.3940685	total: 2.42s	remaining: 6.67s
533:	learn: 0.3940410	total: 2.43s	remaining: 6.67s
534:	learn: 0.3939981	total: 2.44s	remaining: 6.67s
535:	learn: 0.3939922	total: 2.44s	remaining: 6.66s
536:	learn: 0.3939091	total: 2.44s	remaining: 6.66s
537:	learn: 0.3938867	total: 2.45s	remaining: 6.65s
538:	learn: 0.3938298	total: 2.45s	remaining: 6.65s
539:	learn: 0.3938058	total: 2.46s	remaining: 6.64s
540:	learn: 0.3937464	total: 2.46s	remaining: 6.64s
541:	learn: 0.3937360	total: 2.46s	remaining: 6.63s
542:	learn: 0.3936993	total: 2.47s	remaining: 6.63s
543:	learn: 0.3936156	total: 2.47s	remaining: 6.62s
544:	learn: 0.3935341	total: 2.48s	remaining: 6.62s
545:	learn: 0.3935062	total: 2.48s	remaining: 6.62s
546:	learn: 0.3934088	total: 2.49s	remaining: 6.61s
547:	learn: 0.

705:	learn: 0.3851387	total: 3.19s	remaining: 5.86s
706:	learn: 0.3850697	total: 3.2s	remaining: 5.85s
707:	learn: 0.3850431	total: 3.2s	remaining: 5.85s
708:	learn: 0.3850005	total: 3.21s	remaining: 5.84s
709:	learn: 0.3849171	total: 3.21s	remaining: 5.84s
710:	learn: 0.3849089	total: 3.22s	remaining: 5.83s
711:	learn: 0.3848599	total: 3.22s	remaining: 5.83s
712:	learn: 0.3848286	total: 3.23s	remaining: 5.82s
713:	learn: 0.3847659	total: 3.23s	remaining: 5.82s
714:	learn: 0.3846788	total: 3.23s	remaining: 5.81s
715:	learn: 0.3846462	total: 3.24s	remaining: 5.81s
716:	learn: 0.3845960	total: 3.24s	remaining: 5.8s
717:	learn: 0.3845069	total: 3.25s	remaining: 5.8s
718:	learn: 0.3844410	total: 3.25s	remaining: 5.79s
719:	learn: 0.3844176	total: 3.26s	remaining: 5.79s
720:	learn: 0.3843381	total: 3.26s	remaining: 5.79s
721:	learn: 0.3842597	total: 3.27s	remaining: 5.78s
722:	learn: 0.3842044	total: 3.27s	remaining: 5.78s
723:	learn: 0.3841654	total: 3.27s	remaining: 5.77s
724:	learn: 0.38

...

1208:	learn: 0.3631753	total: 5.59s	remaining: 3.66s
1209:	learn: 0.3631495	total: 5.6s	remaining: 3.65s
1210:	learn: 0.3631138	total: 5.6s	remaining: 3.65s
1211:	learn: 0.3630817	total: 5.61s	remaining: 3.65s
1212:	learn: 0.3630368	total: 5.61s	remaining: 3.64s
1213:	learn: 0.3630163	total: 5.62s	remaining: 3.64s
1214:	learn: 0.3629758	total: 5.62s	remaining: 3.63s
1215:	learn: 0.3629391	total: 5.63s	remaining: 3.63s
1216:	learn: 0.3629041	total: 5.63s	remaining: 3.63s
1217:	learn: 0.3628445	total: 5.64s	remaining: 3.62s
1218:	learn: 0.3628223	total: 5.64s	remaining: 3.62s
1219:	learn: 0.3627294	total: 5.65s	remaining: 3.61s
1220:	learn: 0.3626753	total: 5.65s	remaining: 3.61s
1221:	learn: 0.3626627	total: 5.66s	remaining: 3.6s
1222:	learn: 0.3626184	total: 5.66s	remaining: 3.6s
1223:	learn: 0.3625964	total: 5.67s	remaining: 3.59s
1224:	learn: 0.3625150	total: 5.67s	remaining: 3.59s
1225:	learn: 0.3624527	total: 5.68s	remaining: 3.58s
1226:	learn: 0.3624422	total: 5.68s	remaining: 3.5

1370:	learn: 0.3567242	total: 6.39s	remaining: 2.93s
1371:	learn: 0.3566902	total: 6.4s	remaining: 2.93s
1372:	learn: 0.3566324	total: 6.4s	remaining: 2.92s
1373:	learn: 0.3565493	total: 6.41s	remaining: 2.92s
1374:	learn: 0.3565203	total: 6.42s	remaining: 2.92s
1375:	learn: 0.3564869	total: 6.43s	remaining: 2.91s
1376:	learn: 0.3564507	total: 6.43s	remaining: 2.91s
1377:	learn: 0.3563936	total: 6.44s	remaining: 2.9s
1378:	learn: 0.3563530	total: 6.44s	remaining: 2.9s
1379:	learn: 0.3563109	total: 6.45s	remaining: 2.9s
1380:	learn: 0.3562345	total: 6.45s	remaining: 2.89s
1381:	learn: 0.3562093	total: 6.46s	remaining: 2.89s
1382:	learn: 0.3561769	total: 6.46s	remaining: 2.88s
1383:	learn: 0.3561619	total: 6.47s	remaining: 2.88s
1384:	learn: 0.3561041	total: 6.47s	remaining: 2.87s
1385:	learn: 0.3560809	total: 6.48s	remaining: 2.87s
1386:	learn: 0.3560526	total: 6.48s	remaining: 2.86s
1387:	learn: 0.3560432	total: 6.49s	remaining: 2.86s
1388:	learn: 0.3559973	total: 6.49s	remaining: 2.85

1536:	learn: 0.3508275	total: 7.19s	remaining: 2.16s
1537:	learn: 0.3507599	total: 7.19s	remaining: 2.16s
1538:	learn: 0.3507371	total: 7.2s	remaining: 2.15s
1539:	learn: 0.3507333	total: 7.2s	remaining: 2.15s
1540:	learn: 0.3507179	total: 7.21s	remaining: 2.15s
1541:	learn: 0.3506964	total: 7.21s	remaining: 2.14s
1542:	learn: 0.3506518	total: 7.22s	remaining: 2.14s
1543:	learn: 0.3505868	total: 7.22s	remaining: 2.13s
1544:	learn: 0.3505831	total: 7.22s	remaining: 2.13s
1545:	learn: 0.3505159	total: 7.23s	remaining: 2.12s
1546:	learn: 0.3504722	total: 7.23s	remaining: 2.12s
1547:	learn: 0.3504258	total: 7.24s	remaining: 2.11s
1548:	learn: 0.3503976	total: 7.24s	remaining: 2.11s
1549:	learn: 0.3503614	total: 7.25s	remaining: 2.1s
1550:	learn: 0.3503555	total: 7.25s	remaining: 2.1s
1551:	learn: 0.3503356	total: 7.26s	remaining: 2.1s
1552:	learn: 0.3502765	total: 7.26s	remaining: 2.09s
1553:	learn: 0.3502150	total: 7.27s	remaining: 2.08s
1554:	learn: 0.3501821	total: 7.27s	remaining: 2.08

1694:	learn: 0.3453905	total: 7.99s	remaining: 1.44s
1695:	learn: 0.3453711	total: 8s	remaining: 1.43s
1696:	learn: 0.3452912	total: 8s	remaining: 1.43s
1697:	learn: 0.3452119	total: 8.01s	remaining: 1.42s
1698:	learn: 0.3451824	total: 8.01s	remaining: 1.42s
1699:	learn: 0.3451633	total: 8.02s	remaining: 1.41s
1700:	learn: 0.3451288	total: 8.02s	remaining: 1.41s
1701:	learn: 0.3450637	total: 8.03s	remaining: 1.41s
1702:	learn: 0.3450224	total: 8.03s	remaining: 1.4s
1703:	learn: 0.3449655	total: 8.04s	remaining: 1.4s
1704:	learn: 0.3449381	total: 8.04s	remaining: 1.39s
1705:	learn: 0.3449203	total: 8.05s	remaining: 1.39s
1706:	learn: 0.3448851	total: 8.05s	remaining: 1.38s
1707:	learn: 0.3448394	total: 8.06s	remaining: 1.38s
1708:	learn: 0.3448356	total: 8.06s	remaining: 1.37s
1709:	learn: 0.3448189	total: 8.07s	remaining: 1.37s
1710:	learn: 0.3448006	total: 8.07s	remaining: 1.36s
1711:	learn: 0.3447864	total: 8.08s	remaining: 1.36s
1712:	learn: 0.3447536	total: 8.08s	remaining: 1.35s
1

1889:	learn: 0.3394991	total: 9s	remaining: 524ms
1890:	learn: 0.3394786	total: 9.01s	remaining: 519ms
1891:	learn: 0.3394631	total: 9.01s	remaining: 514ms
1892:	learn: 0.3394559	total: 9.02s	remaining: 510ms
1893:	learn: 0.3394293	total: 9.02s	remaining: 505ms
1894:	learn: 0.3393936	total: 9.03s	remaining: 500ms
1895:	learn: 0.3393793	total: 9.03s	remaining: 495ms
1896:	learn: 0.3393669	total: 9.04s	remaining: 491ms
1897:	learn: 0.3393393	total: 9.04s	remaining: 486ms
1898:	learn: 0.3393257	total: 9.05s	remaining: 481ms
1899:	learn: 0.3393063	total: 9.05s	remaining: 476ms
1900:	learn: 0.3392785	total: 9.05s	remaining: 472ms
1901:	learn: 0.3392638	total: 9.06s	remaining: 467ms
1902:	learn: 0.3392358	total: 9.06s	remaining: 462ms
1903:	learn: 0.3392100	total: 9.07s	remaining: 457ms
1904:	learn: 0.3391993	total: 9.07s	remaining: 452ms
1905:	learn: 0.3391797	total: 9.08s	remaining: 448ms
1906:	learn: 0.3391290	total: 9.08s	remaining: 443ms
1907:	learn: 0.3391107	total: 9.09s	remaining: 43

In [311]:
roc_auc_score(Y_test, pred)

0.9086122403666718

In [328]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score

In [329]:
paramgrid = ParameterGrid({'iterations':[10, 20, 50, 100, 200],
                           'max_depth':[4, 5, 6, 8, 10],})

In [330]:
max_accuracy = 0

for parameters in paramgrid:
    model = CatBoostClassifier(iterations=parameters['iterations'],
                            max_depth=parameters['max_depth'])
    model.fit(X_train, Y_train, cat_features=cat_cols)
    pred = model.predict(X_test)
    if accuracy_score(Y_test, pred) > max_accuracy:
        max_accuracy = accuracy_score(Y_test, pred)
        best_model = model

Learning rate set to 0.5
0:	learn: 0.5847815	total: 5.69ms	remaining: 51.2ms
1:	learn: 0.5267533	total: 10.8ms	remaining: 43.2ms
2:	learn: 0.5008745	total: 14.8ms	remaining: 34.6ms
3:	learn: 0.4809190	total: 17.1ms	remaining: 25.6ms
4:	learn: 0.4690416	total: 19.4ms	remaining: 19.4ms
5:	learn: 0.4601893	total: 21.2ms	remaining: 14.1ms
6:	learn: 0.4536739	total: 23ms	remaining: 9.86ms
7:	learn: 0.4468282	total: 24.5ms	remaining: 6.13ms
8:	learn: 0.4400695	total: 26.5ms	remaining: 2.94ms
9:	learn: 0.4368715	total: 28ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.5731386	total: 1.7ms	remaining: 15.3ms
1:	learn: 0.5262058	total: 3.28ms	remaining: 13.1ms
2:	learn: 0.4910549	total: 4.73ms	remaining: 11ms
3:	learn: 0.4718377	total: 6.52ms	remaining: 9.78ms
4:	learn: 0.4566204	total: 8.09ms	remaining: 8.09ms
5:	learn: 0.4473287	total: 9.6ms	remaining: 6.4ms
6:	learn: 0.4409493	total: 10.9ms	remaining: 4.68ms
7:	learn: 0.4352811	total: 12.2ms	remaining: 3.06ms
8:	learn: 0.4314699	total:

Learning rate set to 0.386767
0:	learn: 0.5939410	total: 1.65ms	remaining: 80.9ms
1:	learn: 0.5446989	total: 3.16ms	remaining: 75.8ms
2:	learn: 0.5088253	total: 4.78ms	remaining: 75ms
3:	learn: 0.4892955	total: 6.13ms	remaining: 70.5ms
4:	learn: 0.4724408	total: 7.42ms	remaining: 66.8ms
5:	learn: 0.4637450	total: 8.71ms	remaining: 63.8ms
6:	learn: 0.4529295	total: 10ms	remaining: 61.7ms
7:	learn: 0.4459018	total: 11.4ms	remaining: 59.9ms
8:	learn: 0.4410024	total: 12.6ms	remaining: 57.6ms
9:	learn: 0.4359477	total: 13.9ms	remaining: 55.5ms
10:	learn: 0.4344238	total: 14.8ms	remaining: 52.3ms
11:	learn: 0.4308500	total: 16ms	remaining: 50.7ms
12:	learn: 0.4285873	total: 17.2ms	remaining: 49ms
13:	learn: 0.4255814	total: 18.4ms	remaining: 47.4ms
14:	learn: 0.4225880	total: 19.7ms	remaining: 46ms
15:	learn: 0.4194059	total: 21ms	remaining: 44.5ms
16:	learn: 0.4170689	total: 22.3ms	remaining: 43.2ms
17:	learn: 0.4160032	total: 23.6ms	remaining: 42ms
18:	learn: 0.4152252	total: 25ms	remaini

24:	learn: 0.3793215	total: 197ms	remaining: 197ms
25:	learn: 0.3772122	total: 206ms	remaining: 190ms
26:	learn: 0.3746671	total: 219ms	remaining: 186ms
27:	learn: 0.3725045	total: 227ms	remaining: 179ms
28:	learn: 0.3719507	total: 236ms	remaining: 171ms
29:	learn: 0.3706595	total: 244ms	remaining: 163ms
30:	learn: 0.3696771	total: 253ms	remaining: 155ms
31:	learn: 0.3661551	total: 262ms	remaining: 147ms
32:	learn: 0.3642015	total: 271ms	remaining: 140ms
33:	learn: 0.3627630	total: 280ms	remaining: 132ms
34:	learn: 0.3602859	total: 289ms	remaining: 124ms
35:	learn: 0.3590662	total: 299ms	remaining: 116ms
36:	learn: 0.3570677	total: 309ms	remaining: 108ms
37:	learn: 0.3564279	total: 318ms	remaining: 100ms
38:	learn: 0.3560279	total: 328ms	remaining: 92.4ms
39:	learn: 0.3553169	total: 338ms	remaining: 84.6ms
40:	learn: 0.3546748	total: 348ms	remaining: 76.4ms
41:	learn: 0.3520420	total: 357ms	remaining: 68ms
42:	learn: 0.3518861	total: 367ms	remaining: 59.8ms
43:	learn: 0.3492572	total: 

Learning rate set to 0.204835
0:	learn: 0.6275912	total: 2.06ms	remaining: 204ms
1:	learn: 0.5872545	total: 3.86ms	remaining: 189ms
2:	learn: 0.5581623	total: 5.51ms	remaining: 178ms
3:	learn: 0.5328978	total: 7.23ms	remaining: 174ms
4:	learn: 0.5168289	total: 8.86ms	remaining: 168ms
5:	learn: 0.5028245	total: 10.7ms	remaining: 168ms
6:	learn: 0.4905449	total: 12.4ms	remaining: 165ms
7:	learn: 0.4803460	total: 13.9ms	remaining: 159ms
8:	learn: 0.4733608	total: 15.6ms	remaining: 158ms
9:	learn: 0.4672414	total: 17.4ms	remaining: 157ms
10:	learn: 0.4608228	total: 18.9ms	remaining: 153ms
11:	learn: 0.4535398	total: 20.3ms	remaining: 149ms
12:	learn: 0.4477438	total: 21.7ms	remaining: 146ms
13:	learn: 0.4433844	total: 23.3ms	remaining: 143ms
14:	learn: 0.4397380	total: 24.7ms	remaining: 140ms
15:	learn: 0.4365713	total: 26.2ms	remaining: 138ms
16:	learn: 0.4334589	total: 27.6ms	remaining: 135ms
17:	learn: 0.4312110	total: 29ms	remaining: 132ms
18:	learn: 0.4291496	total: 30.4ms	remaining: 

80:	learn: 0.3692515	total: 197ms	remaining: 46.2ms
81:	learn: 0.3687071	total: 200ms	remaining: 43.8ms
82:	learn: 0.3683440	total: 202ms	remaining: 41.4ms
83:	learn: 0.3676763	total: 205ms	remaining: 39ms
84:	learn: 0.3668413	total: 208ms	remaining: 36.7ms
85:	learn: 0.3666646	total: 215ms	remaining: 35ms
86:	learn: 0.3658703	total: 218ms	remaining: 32.6ms
87:	learn: 0.3652399	total: 221ms	remaining: 30.2ms
88:	learn: 0.3645867	total: 224ms	remaining: 27.7ms
89:	learn: 0.3642623	total: 227ms	remaining: 25.2ms
90:	learn: 0.3639068	total: 229ms	remaining: 22.7ms
91:	learn: 0.3638690	total: 232ms	remaining: 20.2ms
92:	learn: 0.3631656	total: 234ms	remaining: 17.6ms
93:	learn: 0.3631464	total: 236ms	remaining: 15.1ms
94:	learn: 0.3628524	total: 239ms	remaining: 12.6ms
95:	learn: 0.3623607	total: 241ms	remaining: 10ms
96:	learn: 0.3622692	total: 244ms	remaining: 7.53ms
97:	learn: 0.3619495	total: 246ms	remaining: 5.02ms
98:	learn: 0.3616586	total: 248ms	remaining: 2.51ms
99:	learn: 0.36111

...

14:	learn: 0.4618446	total: 58.7ms	remaining: 724ms
15:	learn: 0.4585269	total: 63.5ms	remaining: 730ms
16:	learn: 0.4544110	total: 67.9ms	remaining: 731ms
17:	learn: 0.4513102	total: 71.8ms	remaining: 726ms
18:	learn: 0.4486285	total: 75.9ms	remaining: 723ms
19:	learn: 0.4465009	total: 79.6ms	remaining: 716ms
20:	learn: 0.4430098	total: 83.8ms	remaining: 714ms
21:	learn: 0.4405630	total: 87.8ms	remaining: 711ms
22:	learn: 0.4386250	total: 91ms	remaining: 700ms
23:	learn: 0.4365253	total: 95.6ms	remaining: 701ms
24:	learn: 0.4342017	total: 101ms	remaining: 706ms
25:	learn: 0.4321095	total: 107ms	remaining: 715ms
26:	learn: 0.4306926	total: 113ms	remaining: 727ms
27:	learn: 0.4287862	total: 118ms	remaining: 723ms
28:	learn: 0.4277674	total: 123ms	remaining: 725ms
29:	learn: 0.4266764	total: 127ms	remaining: 721ms
30:	learn: 0.4256848	total: 132ms	remaining: 717ms
31:	learn: 0.4242075	total: 135ms	remaining: 710ms
32:	learn: 0.4227205	total: 139ms	remaining: 705ms
33:	learn: 0.4215460	to

Learning rate set to 0.108483
0:	learn: 0.6406339	total: 5.2ms	remaining: 1.03s
1:	learn: 0.6024386	total: 10.1ms	remaining: 1000ms
2:	learn: 0.5693169	total: 15ms	remaining: 983ms
3:	learn: 0.5439903	total: 20.5ms	remaining: 1s
4:	learn: 0.5280307	total: 24.8ms	remaining: 967ms
5:	learn: 0.5147444	total: 29.7ms	remaining: 961ms
6:	learn: 0.5038069	total: 34.1ms	remaining: 940ms
7:	learn: 0.4918011	total: 38.5ms	remaining: 925ms
8:	learn: 0.4833963	total: 43.1ms	remaining: 914ms
9:	learn: 0.4765305	total: 47.3ms	remaining: 898ms
10:	learn: 0.4695267	total: 51.5ms	remaining: 885ms
11:	learn: 0.4639073	total: 56ms	remaining: 878ms
12:	learn: 0.4588939	total: 61.2ms	remaining: 880ms
13:	learn: 0.4541698	total: 66.9ms	remaining: 888ms
14:	learn: 0.4496081	total: 71.7ms	remaining: 885ms
15:	learn: 0.4461998	total: 76.2ms	remaining: 876ms
16:	learn: 0.4438669	total: 80.5ms	remaining: 866ms
17:	learn: 0.4415388	total: 84.8ms	remaining: 857ms
18:	learn: 0.4393474	total: 88.7ms	remaining: 845ms

173:	learn: 0.3554222	total: 798ms	remaining: 119ms
174:	learn: 0.3550137	total: 803ms	remaining: 115ms
175:	learn: 0.3544827	total: 808ms	remaining: 110ms
176:	learn: 0.3542804	total: 813ms	remaining: 106ms
177:	learn: 0.3540895	total: 818ms	remaining: 101ms
178:	learn: 0.3539662	total: 823ms	remaining: 96.5ms
179:	learn: 0.3537775	total: 828ms	remaining: 92ms
180:	learn: 0.3536615	total: 832ms	remaining: 87.3ms
181:	learn: 0.3533530	total: 838ms	remaining: 82.8ms
182:	learn: 0.3531378	total: 843ms	remaining: 78.3ms
183:	learn: 0.3527184	total: 848ms	remaining: 73.7ms
184:	learn: 0.3521910	total: 853ms	remaining: 69.1ms
185:	learn: 0.3519267	total: 856ms	remaining: 64.5ms
186:	learn: 0.3515087	total: 860ms	remaining: 59.8ms
187:	learn: 0.3512549	total: 865ms	remaining: 55.2ms
188:	learn: 0.3510607	total: 869ms	remaining: 50.6ms
189:	learn: 0.3509354	total: 874ms	remaining: 46ms
190:	learn: 0.3507748	total: 878ms	remaining: 41.4ms
191:	learn: 0.3505481	total: 883ms	remaining: 36.8ms
19

152:	learn: 0.3222401	total: 1.04s	remaining: 321ms
153:	learn: 0.3221195	total: 1.05s	remaining: 314ms
154:	learn: 0.3216192	total: 1.06s	remaining: 307ms
155:	learn: 0.3212533	total: 1.06s	remaining: 300ms
156:	learn: 0.3205188	total: 1.07s	remaining: 293ms
157:	learn: 0.3201163	total: 1.08s	remaining: 287ms
158:	learn: 0.3199344	total: 1.08s	remaining: 280ms
159:	learn: 0.3196957	total: 1.09s	remaining: 274ms
160:	learn: 0.3195726	total: 1.1s	remaining: 267ms
161:	learn: 0.3187267	total: 1.11s	remaining: 261ms
162:	learn: 0.3183473	total: 1.12s	remaining: 255ms
163:	learn: 0.3175471	total: 1.13s	remaining: 249ms
164:	learn: 0.3170389	total: 1.15s	remaining: 243ms
165:	learn: 0.3168582	total: 1.16s	remaining: 238ms
166:	learn: 0.3165262	total: 1.17s	remaining: 232ms
167:	learn: 0.3161355	total: 1.18s	remaining: 226ms
168:	learn: 0.3156422	total: 1.2s	remaining: 219ms
169:	learn: 0.3153844	total: 1.21s	remaining: 213ms
170:	learn: 0.3145459	total: 1.22s	remaining: 206ms
171:	learn: 0.

114:	learn: 0.2982889	total: 1.44s	remaining: 1.06s
115:	learn: 0.2973105	total: 1.46s	remaining: 1.05s
116:	learn: 0.2966057	total: 1.48s	remaining: 1.05s
117:	learn: 0.2955915	total: 1.5s	remaining: 1.04s
118:	learn: 0.2939542	total: 1.52s	remaining: 1.04s
119:	learn: 0.2936295	total: 1.55s	remaining: 1.03s
120:	learn: 0.2934090	total: 1.57s	remaining: 1.02s
121:	learn: 0.2929807	total: 1.59s	remaining: 1.02s
122:	learn: 0.2924265	total: 1.61s	remaining: 1.01s
123:	learn: 0.2914405	total: 1.64s	remaining: 1s
124:	learn: 0.2909582	total: 1.66s	remaining: 993ms
125:	learn: 0.2900200	total: 1.67s	remaining: 982ms
126:	learn: 0.2894312	total: 1.69s	remaining: 970ms
127:	learn: 0.2879352	total: 1.7s	remaining: 957ms
128:	learn: 0.2861427	total: 1.71s	remaining: 943ms
129:	learn: 0.2837823	total: 1.73s	remaining: 930ms
130:	learn: 0.2832474	total: 1.74s	remaining: 916ms
131:	learn: 0.2820014	total: 1.75s	remaining: 903ms
132:	learn: 0.2805790	total: 1.76s	remaining: 889ms
133:	learn: 0.279

In [331]:
max_accuracy


0.835632183908046

In [312]:
test_dataframe = pd.read_csv("data/space_test.csv")

tst_df = preprocess_data(test_dataframe)

tst_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   object 
 1   CryoSleep     4277 non-null   bool   
 2   Destination   4277 non-null   object 
 3   Age           4277 non-null   float64
 4   VIP           4277 non-null   bool   
 5   RoomService   4277 non-null   float64
 6   FoodCourt     4277 non-null   float64
 7   ShoppingMall  4277 non-null   float64
 8   Spa           4277 non-null   float64
 9   VRDeck        4277 non-null   float64
 10  Cabin1        4277 non-null   object 
 11  Cabin2        4277 non-null   object 
 12  Cabin3        4277 non-null   object 
dtypes: bool(2), float64(6), object(5)
memory usage: 376.0+ KB


In [332]:
prediction = best_model.predict(tst_df[feat])


In [333]:
diction = {'PassengerId': test_dataframe['PassengerId'],
          "Transported": prediction}
write_df = pd.DataFrame(diction)

In [336]:
write_df.to_csv('./data/Submission1.csv', index=False)


In [337]:
prediction

array([ True, False,  True, ...,  True,  True,  True])